In [1]:
import os
from niceview.utils.mask import sparse_npz_to_array
from niceview.utils.tools import normalize_log_nonzero_to_range
import numpy as np
import cv2

In [2]:
# configurations
DATA_PATH = '../examples/data/'

In [3]:
# TODO: automate this
mask_filtered_relabeled_name = 'gt-iz-p9-rep2-mask-filtered-relabeled.npz'  # variable
img_name = 'gt-iz-p9-rep2-img.tiff'  # variable
img_gis_name = 'gt-iz-p9-rep2-img-gis.tiff'  # variable
img_masked_name = 'gt-iz-p9-rep2-img-masked.png'  # variable
img_masked_gis_name = 'gt-iz-p9-rep2-img-masked-gis.tiff'  # variable
cell_info_name = 'gt-iz-p9-rep2-cells-info.csv'  # variable

# load data
mask_filtered_relabeled = sparse_npz_to_array(os.path.join(DATA_PATH, mask_filtered_relabeled_name))

In [4]:
def mask_overlay_image(img_path, mask, dst_path, overwrite=False):
    """Overlay mask on image and save to file.

    Args:
        img_path (str): path to image.
        mask (np.ndarray): mask array of shape (row, col), background value is 0.
        dst_path (str): destination path.
        overwrite (bool): whether to overwrite existing file.

    Returns:
        str: destination path.
    """
    if os.path.exists(dst_path) and not overwrite:
        print(f'File {dst_path} already exists.')
        return dst_path
    
    # mask image
    mask_img = cv2.cvtColor(mask.astype(np.uint8), cv2.COLOR_BGR2RGB)
    mask_img = cv2.applyColorMap(mask_img, cv2.COLORMAP_JET)
    
    # background image
    bkgd_img = cv2.imread(img_path)

    # place filter
    mask_place = (mask != 0).astype(np.uint8)  # mask locations
    bkgd_place = (mask == 0).astype(np.uint8)  # background locations

    # overlay
    heatmap = mask_place * np.transpose(mask_img, (2, 0, 1))  # mask
    heatmap = heatmap + bkgd_place * np.transpose(bkgd_img, (2, 0, 1))  # background
    heatmap = np.transpose(heatmap, (1, 2, 0))
    cv2.imwrite(dst_path, heatmap)
    return dst_path

In [5]:
# normalize
mask_filtered_relabeled_norm = normalize_log_nonzero_to_range(mask_filtered_relabeled)

In [6]:
# image with mask overlay witout geo-referencing
img_masked_path = mask_overlay_image(
    os.path.join(DATA_PATH, img_name),
    mask_filtered_relabeled_norm,
    os.path.join(DATA_PATH, img_masked_name),
    overwrite=True,
)